# Implementation of LSMC alghorithm 
#### Longstaff & Schwartz paper: https://people.math.ethz.ch/~hjfurrer/teaching/LongstaffSchwartzAmericanOptionsLeastSquareMonteCarlo.pdf 

In [3]:
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [4]:
class LSMC:
    
    def __init__(self, op_type, S0, K, T, N, r, sigma, M):
        self.op_type = op_type
        self.S0 = S0
        self.K = K
        self.T = T
        self.N = N
        self.r = r
        self.sigma = sigma
        self.M = M
        self.dt = self.T / self.N
        self.df = np.exp(-self.r * self.dt)

    def price_paths(self):
        #np.random.seed(1)
        dW=np.random.standard_normal((self.M, self.N+1))*np.sqrt(self.dt)
        S = np.zeros((self.M, self.N+1))
        S[:,0] = self.S0
        for t in range(1, self.N + 1):
            S[:, t] = (S[:, t-1]*np.exp((self.r-0.5*self.sigma**2)*self.dt+self.sigma*dW[:, t-1]))
        return S
        
    def MCpayoff(self,stock_paths,K,option_type):
        if option_type == 'call':
            payoff = np.maximum(stock_paths - K,0)
        else:
            payoff = np.maximum(K - stock_paths,0)
        return payoff

    def price(self):
        self.S=self.price_paths()
        self.payoff = self.MCpayoff(self.S,self.K,self.op_type)
        value_matrix = np.zeros_like(self.payoff)
        value_matrix[:, -1] = self.payoff[:, -1]
        for t in range(self.N - 1, 0 , -1):
            regression = np.polyfit(self.S[:, t], value_matrix[:, t+1] * self.df, 5)
            continuation_value = np.polyval(regression, self.S[:, t])
            value_matrix[:, t] = np.where(self.payoff[:, t] > continuation_value,
                                          self.payoff[:, t],
                                          value_matrix[:, t+1] * self.df)

        return np.sum(value_matrix[:,1] * self.df) / self.M

In [5]:
#option_type, S0, strike, T, M, r, div, sigma, simulations
op_type, S0, K, T, N, r, sigma, M = "put",100.,100.,1.,40,0.01,0.2,100000

AmericanPUT = LSMC(op_type, S0, K, T, N, r, sigma, M)
AmericanPUT.price()

7.41747719081599